# Prototype V2
- Feature
  - Precision Estimation
  - Recall Estimation
  - F-beta Score Calculation

# Overview
- System
  - Two Classes:
    - Events of Interest
      - Following Gaussian Distribution
    - Evemnts of No Interest
      - Following Gaussian Distribution
  - Event Representation
    - A Single Value
- Estimator
  - A classifier to classify the events into two classes
  - Calculate the precision and recall on the fly
  - Calculate the F-beta score on the fly
- Planning
  - If yes, threshold stays the same
  - If no, threshold goes up
- Controller
  - If yes, threshold stays the same
  - If no, threshold goes up
  
# System
## Events Generation
ratio of events of interest: 0.05
ratio of events of no interest: 0.95
### Events of Interest
mean: xx std: xx

### Events of No Interest
mean: xx std: xx
note: take absolute value

## Triggering Mechanism
- above the threshold, trigger the event, log as 1
- below the threshold, do not trigger the event, log as 0

# Estimator
- assume it is 100% accurate, use the ground truth data
- use a container to hold event
  - ground truth label
  - captured or not
  - calculate recall and precision
  - calculate F-beta

# Planning
- if the captured event is classified as 1, then the threshold stays the same
- if the captured event is classified as 0, then the threshold goes up

# Controller
- if the captured event is classified as 1, then the threshold stays the same
- if the captured event is classified as 0, then the threshold goes up

In [5]:
# Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import csv
import random
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Dropout

In [6]:
# Predefined Variables and Constants

# Global Variables
Num_Events = 10000
ratio_interest = 0.05
Num_Events_Interest = int(Num_Events * ratio_interest)
Num_Events_NoInterest = Num_Events - Num_Events_Interest
random_number = 0
random_seed = 2023
random.seed(random_seed)
event_value = 0
event_class = 0

# History Record

# Events: (1) time step, (2) value, (3) ground truth label, (4) captured or not (determined by the triggering mechanism), (5) classified label (determined by the estimator)
Events = np.zeros((Num_Events, 5))

# Thresholds: (1) time step, (2) threshold value
Thresholds_Current = np.zeros((Num_Events, 2)) # columns: time step, threshold value (current)
Thresholds_Target = np.zeros((Num_Events, 2)) # columns: time step, threshold value (target)


In [7]:
# I - System 
# Events of No Interest
mean_0 = 10
std_0 = 3

# Events of Interest
mean_1 = 15
std_1 = 3

# Triggering Mechanism
threshold = 0 # initial value
threshold_target = 1 # initialized randomly, don't be the same as threshold

change_interval = (mean_1 - mean_0)/100 # the interval of threshold change



In [8]:
# II - Estimator
EST_WINDOW_LEN = 500
BETA = 2

Metrics = np.zeros((Num_Events, 7)) # Columns: (1) TP (2) FP (3) TN (4) FN (5) Recall (6) Precision (7) Fbeta - current
Metrics_GT = np.zeros((Num_Events, 7)) # Columns: (1) TP_GT (2) FP_GT (3) TN_GT (4) FN_GT (5) Recall_GT (6) Precision_GT (7) Fbeta_GT - ground truth 

Event_Window = np.zeros((EST_WINDOW_LEN, 5)) # initialization, to be updated on the fly : columns: time step, value, ground truth label, captured or not (determined by the triggering mechanism), classified label (determined by the estimator)

# function definition
def Metrics_Compute(Event_Window):
    # Compute the metrics
    # Input: Events
    # Output: Metrics: Recall, Precision, Fbeta Score
    TP = 0
    FP = 0
    TN = 0
    FN = 0
    Recall = 0
    Precision = 0
    Fbeta = 0
    
    for i in range(Event_Window.shape[0]):
        if Event_Window[i,3] == 1 and Event_Window[i,4] == 1:
            TP += 1
        elif Event_Window[i,3] == 0 and Event_Window[i,4] == 1:
            FN += 1
        elif Event_Window[i,3] == 1 and Event_Window[i,4] == 0:
            FP += 1
        else:
            TN += 1
    # Recall
    if TP + FN == 0:
        Recall = 0
    else:
        Recall = TP/(TP+FN)

    # Precision
    if TP + FP == 0:
        Precision = 0
    else:
        Precision = TP/(TP+FP)
        
    # Fbeta
    if Precision + Recall == 0:
        Fbeta = 0
    else:
        Fbeta = (1+BETA*BETA)*Precision*Recall/(BETA*BETA*Precision+Recall)
        
    return TP, FP, TN, FN, Recall, Precision, Fbeta

def Metrics_GT_Compute(Event_Window):
    # Compute the metrics
    # Input: Events
    # Output: Metrics: Recall, Precision, Fbeta Score
    TP_GT = 0
    FP_GT = 0
    TN_GT = 0
    FN_GT = 0
    Recall_GT = 0
    Precision_GT = 0
    Fbeta_GT = 0
    
    for i in range(EST_WINDOW_LEN):
        if Event_Window[i,2] == 1 and Event_Window[i,3] == 1:
            TP_GT += 1
        elif Event_Window[i,2] == 0 and Event_Window[i,3] == 1:
            FP_GT += 1
        elif Event_Window[i,2] == 1 and Event_Window[i,3] == 0:
            FN_GT += 1
        else:
            TN_GT += 1
    # Recall
    if TP_GT + FN_GT == 0:
        Recall_GT = 0
    else:
        Recall_GT = TP_GT/(TP_GT+FN_GT)

    # Precision
    if TP_GT + FP_GT == 0:
        Precision_GT = 0
    else:
        Precision_GT = TP_GT/(TP_GT+FP_GT)
        
    # Fbeta
    if Precision_GT + Recall_GT == 0:
        Fbeta_GT = 0
    else:
        Fbeta_GT = (1+BETA*BETA)*Precision_GT*Recall_GT/(BETA*BETA*Precision_GT+Recall_GT)
        
    return TP_GT, FP_GT, TN_GT, FN_GT, Recall_GT, Precision_GT, Fbeta_GT



In [9]:
# III - Planner and Controller
# refer to the next cell

In [10]:
# SATM Prototype
# no NN involved in this version

# IV - Main Loop (Simulation)

for i in range (0,Num_Events):
    
    # print progress
    if i % 500 == 0:
        print("Simulation Progress %d/%d" %(i, Num_Events))
    Events[i,0] = i
    
    # System / Process Simulation
    # - generate events and log them
    random_number = random.random()
    if random_number <= ratio_interest:
        # event of interest
        event_class = 1
        Events[i,2] = event_class
        event_value = random.gauss(mean_1, std_1)
        Events[i,1] = event_value
    else:
        # event of no interest
        event_class = 0
        Events[i,2] = event_class 
        event_value = random.gauss(mean_0, std_0)
        event_value = np.abs(event_value)
        Events[i,1] = event_value
    
    # - triggering simulation
    if event_value > threshold:
        Events[i,3] = 1

    # Estimation
    # - classification - for now we assume the classifier is 100% accurate, i.e., the classified label is the same as the ground truth label  
    Events[i,4] = Events[i,2]
    
    # - metrics computation
    if i < EST_WINDOW_LEN:
        Event_Window = Events[0:i,:]
    else:
        Event_Window = Events[i-EST_WINDOW_LEN:i,:]
        
    Metrics[i,:] = Metrics_Compute(Event_Window)
    Metrics_GT[i,:] = Metrics_GT_Compute(Event_Window)
    
    # Planner and Controller
    # - Update the threshold_current
    if Events[i,4] == 1 and Events[i,3] == 0:
        threshold = threshold - change_interval
    elif Events[i,4] == 0 and Events[i,3] == 1:
        threshold = threshold + change_interval
    
    Thresholds_Current[i,0] = i
    Thresholds_Current[i,1] = threshold
    
    # - Update the threshold_target
    if Events[i,2] == 1 and Events[i,3] == 0:
        threshold = threshold + change_interval
    elif Events[i,2] == 0 and Events[i,3] == 1:
        threshold = threshold - change_interval
    
    Thresholds_Target[i,0] = i
    Thresholds_Target[i,1] = threshold
    
    # - Update others
    Events[i,0] = i
    
    # Reset
    threshold = 0

    

Simulation Progress 0/10000


IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
# plot
# plot events with dots, thresholds with lines
plt.figure(figsize=(20,10))
plt.plot(Events[:,0], Events[:,1], c=Events[:,2], marker='o', cmap = 'coolwarm', label = 'Events')
# plot metrics with lines

# labels
plt.xlabel('Time')
plt.ylabel('Event Value')

# title
plt.title('SATM Prototype')

# save figure
plt.savefig('SATM_Prototype.png')

# show figure
plt.show()